In [37]:
import numpy as np
import pandas as pd
import random 
import warnings
from tqdm import tqdm
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_excel("/home/ahmed/Desktop/Online Retail.xlsx")
dfprv=df

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
print(df.shape)
df.isnull().sum()
if df.isnull().sum().sum() >= 1:
    print("Exsit Null Value \nRemoving it..... \nDone!")
    df.dropna(inplace=True)

(541909, 8)
Exsit Null Value 
Removing it..... 
Done!


In [6]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
df['StockCode'] = df['StockCode'].astype(str)
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [8]:
customers = df['CustomerID'].unique().tolist()
print(len(customers))

4372


In [9]:
random.shuffle(customers)

customers_train=[customers[i] for i in range(round(0.9*len(customers)))]

train_df=df[df['CustomerID'].isin(customers_train)]
valid_df=df[~df['CustomerID'].isin(customers_train)]
print("Training set size : {} \nValidation set size :{}".format(len(customers_train),len(customers)-len(customers_train)))

Training set size : 3935 
Validation set size :437


In [10]:
purchases_train=[]

for i in tqdm(customers_train):
    temp=train_df[train_df["CustomerID"]==i]["StockCode"].tolist()
    purchases_train.append(temp)
    

100%|██████████| 3935/3935 [00:09<00:00, 420.37it/s]


In [11]:
purchases_valid=[]

for i in tqdm(valid_df['CustomerID'].unique()):
    temp=valid_df[valid_df["CustomerID"]==i]["StockCode"].tolist()
    purchases_valid.append(temp)


100%|██████████| 437/437 [00:00<00:00, 754.06it/s]


In [30]:
model=Word2Vec(window=10,sg=1,hs=1,negative=10,min_alpha=0.0007,seed=14,alpha=0.03)

In [31]:
model.build_vocab(purchases_train,progress_per=200)

In [32]:
model.train(purchases_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(3661404, 3697660)

In [33]:
model.save("word2vec_2.model")

In [34]:
model.init_sims(replace=True)

In [35]:
print(model)

Word2Vec(vocab=3180, size=100, alpha=0.03)


In [39]:
products = train_df[["StockCode", "Description"]]

products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()
products_dict['84029E']

['RED WOOLLY HOTTIE WHITE HEART.']

In [40]:
def similar_products(v, n = 6):
    
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [41]:
similar_products(model['90019A'])

[('SILVER M.O.P ORBIT DROP EARRINGS', 0.7591123580932617),
 ('DROP DIAMANTE EARRINGS PURPLE', 0.7360589504241943),
 ('AMBER DROP EARRINGS W LONG BEADS', 0.6862317323684692),
 ('BLUE MURANO TWIST BRACELET', 0.6512300968170166),
 ('SILVER M.O.P. ORBIT NECKLACE', 0.650446891784668),
 ('ANT COPPER RED BOUDICCA BRACELET', 0.6411212086677551)]